In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge, Patch
from matplotlib.lines import Line2D
import numpy as np

# Załaduj dane (zakładamy, że masz już warstwy gdf_overlay_regions i gdf_merged)
# gdf_overlay_regions - warstwa poligonowa
# gdf_merged - warstwa punktowa

# Przekształć obie warstwy do układu współrzędnych EPSG:3857 (Web Mercator)
gdf_overlay_regions = gdf_overlay_regions.to_crs(epsg=3857)
gdf_merged = gdf_merged.to_crs(epsg=3857)

# 1. Kartogram dla warstwy poligonowej
fig, ax = plt.subplots(figsize=(12, 8))

# Narysuj poligony z kolorystyką na podstawie kolumny total_power_sum
gdf_overlay_regions.plot(
    column='total_power_sum',  # Kolumna do kolorowania
    cmap='OrRd',              # Paleta kolorów
    legend=True,              # Dodaj legendę
    ax=ax,
    legend_kwds={'label': "Total Power Sum", 'shrink': 0.6}  # Dostosuj legendę
)

# 2. Wykresy kołowe dla warstwy punktowej
for idx, row in gdf_merged.iterrows():
    # Oblicz sumę urban_sum i suburban_sum
    total = row['urban_sum'] + row['suburban_sum']
    
    # Oblicz proporcje
    urban_ratio = row['urban_sum'] / total
    suburban_ratio = row['suburban_sum'] / total
    
    # Współrzędne punktu w układzie EPSG:3857
    x, y = row.geometry.x, row.geometry.y
    
    # Rozmiar koła (można dostosować)
    size = total / 1000  # Przykładowe skalowanie
    
    # Narysuj wykres kołowy
    angles = np.cumsum([0, urban_ratio * 360, suburban_ratio * 360])
    for i in range(len(angles) - 1):
        wedge = Wedge(
            (x, y),                     # Środek koła
            size,                       # Promień
            angles[i],                  # Początkowy kąt
            angles[i + 1],              # Końcowy kąt
            color=['blue', 'orange'][i], # Kolory dla urban i suburban
            alpha=0.7                   # Przezroczystość (0.0 - przezroczyste, 1.0 - nieprzezroczyste)
        )
        ax.add_patch(wedge)

# 3. Dodaj legendę dla kółek
# Tworzymy własne obiekty Patch do legendy
urban_patch = Patch(color='blue', label='Urban Sum', alpha=0.7)
suburban_patch = Patch(color='orange', label='Suburban Sum', alpha=0.7)

# Oblicz maksymalną sumę urban_sum + suburban_sum
max_total = gdf_merged['urban_sum'].add(gdf_merged['suburban_sum']).max()
max_total_rounded = round(max_total / 100) * 100  # Zaokrąglij do 100

# Tworzymy koła do legendy za pomocą Line2D
max_circle = Line2D(
    [0], [0],  # Współrzędne (nieużywane)
    marker='o',  # Koło
    markersize=np.sqrt(max_total_rounded / 1000) * 10,  # Rozmiar koła
    color='gray',
    alpha=0.7,
    label=f'{max_total_rounded} obiektów',
    linestyle='none'  # Bez linii
)
min_circle = Line2D(
    [0], [0],  # Współrzędne (nieużywane)
    marker='o',  # Koło
    markersize=np.sqrt(10 / 1000) * 10,  # Rozmiar koła
    color='gray',
    alpha=0.7,
    label='10 obiektów',
    linestyle='none'  # Bez linii
)

# Dodaj legendę do mapy
ax.legend(
    handles=[urban_patch, suburban_patch, max_circle, min_circle],
    loc='upper right',
    title="Legenda dla kółek\n(Rozmiar zależy od sumy urban_sum + suburban_sum)"
)

# Dodaj tytuł
ax.set_title("Kartogram: Total Power Sum (poligony) + Urban/Suburban Sum (wykresy kołowe)", fontsize=14)

# Wyłącz osie (dla lepszej wizualizacji)
ax.set_axis_off()

# Wyświetl mapę
plt.show()